In [1]:
"""
Import libraries and setup DicomMetaParser object.
"""

import pandas as pd
from pathlib import Path
import sys
Path("output").mkdir(parents=True, exist_ok=True)
sys.path.append("../")

from lib.dicom import DicomMetaParser
from lib.utils import load
from covid_ct.utils import filter_all_types, classify_images, parse_subject

parser = DicomMetaParser("/data/covid_ct", parse_subject)

In [2]:
"""
Run this cell to re-parse DICOM metadata.
"""

df = parser.parse()
df.to_pickle("output/metadata.pkl")
# Uncomment to check dataframe column properties
# parser.check_df(df)

Parsing DICOM metadata:   0%|          | 0/482874 [00:00<?, ?it/s]

In [3]:
"""
Run this cell to generate DICOM paths for creating dataset based on metadata.pkl generated above.
"""

df = pd.read_pickle("output/metadata.pkl")
subject_info = load("input/subject_info.pkl")

# Remove bad (blurry / movement artifacts) and negative subjects
df = df[
    (~df["Subject"].isin(subject_info["bad"])) & (df["Subject"].isin(subject_info["positive"]))
]

# Classify CTs into lung vs med vs localizer Types
df = classify_images(df)

# Select DICOM series (based on SeriesNumber) with minimum slice thickness from each CT Type
df = parser.select_min(df, sub_category=["Type"])

# Only select subjects with all 3 (lung, med, localizer) types of CT
df = filter_all_types(df)

# Check that specified fields are consistent within each DICOM Series
df = parser.check_series_consistency(
    df,
    [
        "ImageOrientationPatient",
        "SeriesDescription",
        "PixelSpacing",
        "ImageType",
    ],
)

# Extract file paths for each unique DICOM series (based on Subject and Type)
paths_df = parser.to_path_list(df)
paths_df.to_pickle("output/ct_paths.pkl")

735/735 subjects has lung + med + localizer
